Parameter Setting

In [1]:
dropoutRatio = 0.0
modelDir = "models/"
model_name = "SVHNdropoutRatio" + str(dropoutRatio) 

Train Model

In [2]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

In [20]:
#connect to gdrive

#from google.colab import drive
#drive.mount('/gdrive')
#%cd /gdrive/My\ Drive/

In [4]:
import scipy.io
mat_train = scipy.io.loadmat("./train_32x32.mat")
mat_test = scipy.io.loadmat("./test_32x32.mat")

In [5]:
import numpy as np
mat_images = mat_train["X"]  
mat_labels = mat_train["y"]

In [6]:
mat_images = mat_images[np.newaxis,...]
mat_images = np.swapaxes(mat_images,0,4).squeeze()

In [7]:
from sklearn.model_selection import train_test_split
train_images, valid_images, train_labels, valid_labels = train_test_split(mat_images, mat_labels, train_size=0.8, random_state=13, shuffle=True)

In [8]:
test_images = mat_test["X"]
test_labels = mat_test["y"]

In [9]:
test_images = test_images[np.newaxis,...]
test_images = np.swapaxes(test_images,0,4).squeeze()

In [10]:
train_images = train_images.astype('float32') / 255 - 0.5
test_images = test_images.astype('float32') / 255 - 0.5
valid_images = valid_images.astype('float32') / 255 - 0.5

In [11]:
np.place(train_labels, train_labels == 10, 0)
np.place(test_labels, test_labels == 10, 0)
np.place(valid_labels, valid_labels == 10, 0)

In [12]:
from keras.utils.np_utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)
valid_labels = to_categorical(valid_labels)

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=train_images.shape[1:], activation = 'relu'))
model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropoutRatio))

model.add(Conv2D(64, (3, 3), padding='same', activation = 'relu'))
model.add(Dropout(dropoutRatio))
model.add(Conv2D(64, (3, 3), activation= 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(dropoutRatio))

model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(dropoutRatio))
model.add(Dense(512, activation = 'relu'))
model.add(Dropout(dropoutRatio))
model.add(Dense(10, activation = 'softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        3

In [15]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [16]:
fit = model.fit(train_images, train_labels,
              epochs=30, batch_size=128,
              validation_data=(valid_images, valid_labels),
              shuffle=True)

Epoch 1/30
458/458 [==============================] - 37s 12ms/step - loss: 1.3341 - accuracy: 0.5455 - val_loss: 0.4038 - val_accuracy: 0.8795
Epoch 2/30
458/458 [==============================] - 5s 10ms/step - loss: 0.3286 - accuracy: 0.9034 - val_loss: 0.3150 - val_accuracy: 0.9075
Epoch 3/30
458/458 [==============================] - 4s 10ms/step - loss: 0.2225 - accuracy: 0.9339 - val_loss: 0.2932 - val_accuracy: 0.9145
Epoch 4/30
458/458 [==============================] - 4s 10ms/step - loss: 0.1576 - accuracy: 0.9537 - val_loss: 0.3185 - val_accuracy: 0.9159
Epoch 5/30
458/458 [==============================] - 4s 10ms/step - loss: 0.1090 - accuracy: 0.9687 - val_loss: 0.3466 - val_accuracy: 0.9235
Epoch 6/30
458/458 [==============================] - 4s 9ms/step - loss: 0.0792 - accuracy: 0.9781 - val_loss: 0.3869 - val_accuracy: 0.9178
Epoch 7/30
458/458 [==============================] - 4s 10ms/step - loss: 0.0589 - accuracy: 0.9820 - val_loss: 0.4243 - val_accuracy: 0.9177

Save Model

In [18]:
modelDir = modelDir if modelDir.endswith("/") else modelDir + "/"
model.save( modelDir + model_name+".h5", include_optimizer = False)